In [ ]:
# `한글 지원 라이브러리 설치`

!pip install koreanize-matplotlib
import koreanize_matplotlib

# 뉴스 데이터 크롤링

## 네이버 API 시도..
- 특정기간 검색이 안 돼요ㅠ

In [ ]:
import os
import sys
import json
import urllib.request
import pandas as pd

# 네이버 API 인증 정보
client_id = "wwu6QvE49Xd8miI8EHKc"
client_secret = "gdobGHWv_J"

# 검색어 인코딩
encText = urllib.parse.quote("고려아연")
#disPlay = urllib.parse.quote("5") #한번에 표시할 검색 개수. 기본값10, 최대100
#staRt = urllib.parse.quote("500") #검색 시작위치. 기본값1, 최대1000
disPlay = "15"
staRt = "50"
soRt = urllib.parse.quote("date") #sim:정확도순, date:최신날짜순

url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&" + disPlay + "&" + staRt + "&" + soRt


# API 요청
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if rescode == 200:
    response_body = response.read()
    data = json.loads(response_body.decode('utf-8'))  # JSON 데이터를 파싱
    items = data.get('items', [])  # 'items' 키에서 데이터 가져오기

    # DataFrame 생성
    df = pd.DataFrame(items)
    df
else:
    print("Error Code:", rescode)

In [ ]:
df[['title', 'description', 'pubDate']]

In [ ]:
df

## chromedriver 시도 - 성공

https://somjang.tistory.com/entry/MAC-OSX-개발자를-확인할-수-없기-때문에-chromedriver을를-열-수-없습니다-해결-방법

### 크롤링 테스트

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# ChromeDriver 경로 설정
driver_path = "/Users/leesangwon/Desktop/chromedriver-mac-arm64/chromedriver"  # 여기에 ChromeDriver의 경로를 입력
service = Service(driver_path)

# 웹드라이버 초기화
driver = webdriver.Chrome(service=service)

In [ ]:
# 한글 인코딩
import urllib.parse

text = "[속보] 우크라"

# URL 인코딩
encoded_text = urllib.parse.quote(text)

year = "2024"
month = "02"
start_d = "15"
end_d = "31"

url = f"https://search.naver.com/search.naver?where=news&query={encoded_text}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={year}.{month}.{start_d}&de={year}.{month}.{end_d}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{year}{month}{start_d}to{year}{month}{end_d}&is_sug_officeid=0&office_category=1&service_area=0"
print(url)

# 웹사이트 열기
driver.get(url)

In [ ]:
driver.quit()

### 사용할 날짜 리스트

In [ ]:
# 연도
year = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

# 월
month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# 1~8일   # 9~15일   # 16~23일   # 24~31일
start_d = ["01", "09", "16", "24"]
end_d = ["08", "15", "23", "31"]

# 총 480개 조합

### 최종 크롤링 코드

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import urllib.parse
import time
import pandas as pd

# 데이터프레임 리스트 초기화
all_data = []

#####################################################
# 검색어, 연도, 월, 날짜 리스트 설정
text = "속보 우크라"
year = ['2022']
month = ['01', '02', '03']
start_d = ["01", "09", "16", "24"]
end_d = ["08", "15", "23", "31"]
#####################################################


# ChromeDriver 경로 설정
driver_path = "/Users/leesangwon/Desktop/chromedriver-mac-arm64/chromedriver"  # 여기에 ChromeDriver의 경로를 입력
service = Service(driver_path)

# 웹드라이버 초기화
driver = webdriver.Chrome(service=service)

# 연도, 월, 날짜를 순회하며 크롤링
for y in year:
    for m in month:
        for sd, ed in zip(start_d, end_d):
            # URL 생성
            encoded_text = urllib.parse.quote(text)  # 검색어를 url 인코딩
            url = f"https://search.naver.com/search.naver?where=news&query={encoded_text}&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={y}.{m}.{sd}&de={y}.{m}.{ed}&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{y}{m}{sd}to{y}{m}{ed}&is_sug_officeid=0&office_category=1&service_area=0"

            # 페이지 열기
            driver.get(url)

            # 페이지 끝까지 스크롤
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # 기사 제목, 날짜, 스크립트 크롤링
            titles = driver.find_elements(By.CSS_SELECTOR, ".news_tit")
            dates = driver.find_elements(By.CSS_SELECTOR, ".info_group > span")
            scripts = driver.find_elements(By.CSS_SELECTOR, ".dsc_txt_wrap")
            press = driver.find_elements(By.CSS_SELECTOR, ".info_group > a.press")

            # 데이터 저장
            for title, date, script, p in zip(titles, dates, scripts, press):
                all_data.append({
                    "Title": title.text,
                    "Date": date.text,
                    "Press": p.text,
                    "Description": script.text
                })

# 데이터프레임 생성 및 저장
df1 = pd.DataFrame(all_data)
df1.to_csv(f"/Users/leesangwon/Downloads/{text}_news_data.csv", index=False)

# 드라이버 종료
driver.quit()

# 관련 기사 급증하는 일자 특정

In [ ]:
import pandas as pd
df = pd.read_csv('/Users/leesangwon/Downloads/news_data_2.csv')
df

In [ ]:
date_group = df.groupby('Date').size().reset_index(name='Count')
date_group.head()

In [ ]:
import matplotlib.pyplot as plt

# 날짜 데이터를 시계열로 변환
date_group['Date'] = pd.to_datetime(date_group['Date'], errors='coerce')
date_group = date_group.dropna(subset=['Date'])  # 변환에 실패한 날짜 제거

# 꺾은선 그래프 시각화
plt.figure(figsize=(14, 6))
plt.plot(date_group['Date'], date_group['Count'], marker='o')
plt.title('"[속보] 우크라" 일별 보도 빈도수', fontsize=20)
plt.xlabel('일자', fontsize=12)
plt.ylabel('횟수', fontsize=12)
plt.xticks(rotation=45)
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# 2022년 2월 24일 뉴스 보도가 급증했음 = 러시아의 침공날짜!
# 보도가 나온 날짜 vs. 보도가 떠들석하다가 점점 사그라드는 시점(약 3월 7일 정도)의 주식 수익률을 알아보자

# 특정된 일자 기준으로 주가 수익률 확인 및 라벨링

In [ ]:
import pandas as pd
import yfinance as yf

start = '2024-10-10'
end = '2024-10-15'

tickers = pd.read_csv(f'/Users/leesangwon/Downloads/kosdaq_tickers_test.csv', encoding="utf-8")
# 티커 칼럼 리스트화
tickers_list = tickers['티커'].tolist() if '티커' in tickers.columns else tickers.iloc[:, 0].tolist()
symbol = tickers_list

stock = yf.download(symbol, start, end)
stock = stock['Adj Close']

stock

In [ ]:
# 인덱스를 datetime 형식으로 변환
stock.index = pd.to_datetime(stock.index)

before = stock.loc["2022-02-24"]
after = stock.loc["2022-03-07"]

# 수익률 계산
returns = round((after - before) / before * 100, 3)

# 수익률 데이터를 데이터프레임으로 변환
returns_df = pd.DataFrame(returns, columns=["Return"])

# 인덱스를 칼럼으로 변환하여 "Ticker"라는 새 칼럼 생성
returns_df.reset_index(inplace=True)
returns_df.rename(columns={"index": "Ticker"}, inplace=True)

# 티커와 종목명 간의 매핑 딕셔너리 생성
ticker_to_name = dict(zip(tickers['티커'], tickers['종목명']))

# 티커를 종목명으로 대치
returns_df["Ticker"] = returns_df["Ticker"].map(ticker_to_name)

# 수익률을 내림차순으로 정렬
returns_df.sort_values(by="Return", ascending=False, inplace=True)

# 결과 출력
returns_df

In [ ]:
a = returns_df.loc[returns_df["Ticker"] == "카카오"]
a

In [ ]:
driver.quit()

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

# 페이지 끝까지 스크롤
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 페이지가 로드되도록 대기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 기사 제목, 날짜, 스크립트 크롤링
titles = driver.find_elements(By.CSS_SELECTOR, ".news_tit")  # 기사 제목
dates = driver.find_elements(By.CSS_SELECTOR, ".info_group > span")  # 기사 날짜
scripts = driver.find_elements(By.CSS_SELECTOR, ".dsc_txt_wrap")  # 기사 요약

# 데이터 저장
data = []
for title, date, script in zip(titles, dates, scripts):
    data.append({
        "Title": title.text,
        "Date": date.text,
        "Description": script.text
    })

# 데이터프레임 생성
df = pd.DataFrame(data)
df

In [5]:
import pandas as pd
import yfinance as yf

# 티커 데이터 읽기
tickers_df = pd.read_csv('/Users/leesangwon/Documents/태블로대시보드용_금융데이터/이상치66개_재무정보.csv', encoding="utf-8")
tickers = tickers_df['Ticker'].dropna().tolist()

# 날짜 범위 설정
start_date = "2024-10-01"
end_date = "2024-10-31"

# 종가 데이터를 저장할 데이터프레임
closing_prices = pd.DataFrame()

# Yahoo Finance 데이터 가져오기
for ticker in tickers:
    try:
        data = yf.download(ticker, start=start_date, end=end_date)[['Close']]
        data.columns = [ticker]  # 티커명을 컬럼명으로 변경
        closing_prices = pd.concat([closing_prices, data], axis=1)  # 데이터프레임 병합
    except Exception as e:
        print(f"Error processing {ticker}: {e}")


closing_prices

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,041440.KS,226400.KS,003160.KS,053280.KS,028100.KS,006740.KS,007280.KS,008730.KS,010690.KS,016450.KS,...,314130.KS,315640.KS,424960.KS,317770.KS,319400.KS,399720.KS,403870.KS,418470.KS,000520.KS,445680.KS
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-02,7380.0,6280.0,14700.0,5150.0,12090.0,1353.0,1497.0,23050.0,8920.0,4505.0,...,3455.0,5530.0,10890.0,2830.0,2890.0,42050.0,31750.0,14790.0,15440.0,17990.0
2024-10-04,7180.0,5990.0,15590.0,5110.0,12090.0,1344.0,1498.0,22300.0,8870.0,4525.0,...,3455.0,5560.0,10460.0,2730.0,2875.0,42650.0,31850.0,14800.0,15800.0,18800.0
2024-10-07,7400.0,6100.0,16110.0,5140.0,12090.0,1354.0,1503.0,22600.0,9220.0,4535.0,...,3405.0,5600.0,11200.0,2655.0,2975.0,44250.0,32250.0,15100.0,16160.0,18960.0
2024-10-08,7410.0,5680.0,15840.0,5180.0,12110.0,1331.0,1472.0,22650.0,9210.0,4505.0,...,3350.0,5580.0,10580.0,2640.0,2990.0,43400.0,31100.0,15080.0,17060.0,19020.0
2024-10-10,7640.0,5710.0,15730.0,4915.0,12170.0,1312.0,1425.0,22800.0,9450.0,4500.0,...,3270.0,5560.0,10200.0,2605.0,2985.0,42200.0,30650.0,15110.0,15850.0,19340.0
2024-10-11,7950.0,6100.0,16960.0,6380.0,12510.0,1359.0,1500.0,25400.0,10110.0,5850.0,...,3420.0,5750.0,11160.0,2750.0,3120.0,43800.0,33200.0,18680.0,16640.0,21850.0
2024-10-14,7890.0,6100.0,17800.0,6380.0,12510.0,1385.0,1500.0,25400.0,10110.0,5850.0,...,4445.0,5940.0,11530.0,2840.0,3145.0,43900.0,33150.0,19500.0,16640.0,21850.0
2024-10-15,8140.0,6260.0,19540.0,8750.0,12800.0,1339.0,1422.0,28800.0,10610.0,5930.0,...,4010.0,5960.0,12000.0,3215.0,3160.0,43200.0,33450.0,16360.0,17700.0,22800.0
2024-10-16,8290.0,6350.0,19370.0,7750.0,12830.0,1335.0,1478.0,26950.0,10570.0,5410.0,...,4110.0,6280.0,11700.0,3355.0,3160.0,40850.0,31650.0,15420.0,17180.0,24400.0


In [6]:
# CSV 파일로 저장
output_path = '/Users/leesangwon/Downloads/이상치66개_10월_종가.csv'
closing_prices.to_csv(output_path, index_label='Date')

print(f"데이터가 저장되었습니다: {output_path}")

데이터가 저장되었습니다: /Users/leesangwon/Downloads/이상치66개_10월_종가.csv
